# A/B Testing Part 2

In [5]:
import numpy as np
import pandas as pd

from scipy import stats
import seaborn as sns

# Objectives

- Conduct an A/B test in Python
- Interpret the results of the A/B tests for a stakeholder

# Example Together

## Question

We have data about whether customers completed sales transactions, segregated by the type of ad banners to which the customers were exposed.

The question we want to answer is whether there was any difference in sales "conversions" between desktop customers who saw the sneakers banner and desktop customers who saw the accessories banner in the month of May 2019.

## Considerations

What would we need to consider when designing our experiment?

Might include:

- Who is it that we're including in our test?
- How big of an effect would make it "worth" us seeing?
    - This can affect sample size
    - This can give context of a statistically significant result
- Other biases or "gotchas"

## Loading the Data

First let's download the data from [kaggle](https://www.kaggle.com/podsyp/how-to-do-product-analytics) via the release page of this repo: https://github.com/flatiron-school/ds-ab_testing/releases 

The code below will load it into our DataFrame:

In [6]:
# This will download the data from online so it can take
# some time (but relatively small download)

df = pd.read_csv('https://github.com/flatiron-school/ds-ab_testing/releases/download/v1.2/products_small.csv')

> Let's take a look while we're at it

In [7]:
df.head()

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,title,target
0,4122928,3e6c5e89fdddcaee0eed210ec2c9cadf,90d58d967eb72656e86059ec6f208092,2fdc16a09e0016555dd4da4a3fe84414,accessories,desktop,2019-03-06 08:42:47,banner_show,0
1,564306,feed6203517d3abf6aab13761633174b,08703dab1f004eabba25aacb7f0e5484,6b0a902b9b73d5a158d0119d6feb38ac,sneakers,mobile,2019-04-19 18:30:45,banner_show,0
2,1872289,e33d5d7941edc281646aa37763729771,bdf1d25697e21419901c94fabdafad15,9ddb7315c4357929931b48f2b3d11c62,company,mobile,2019-01-20 17:20:10,banner_show,0
3,3616779,7c4caa8d508fa7c3bbc25f35cdd9168a,8d2f23a732c9527d95678088a3bac122,1f86cd0bea31d54a5b511b42fd19401a,sneakers,mobile,2019-02-20 09:38:32,banner_show,0
4,5871482,12874b29bde8bbd43fb2b95735caf9e6,5a22604f8f31ae98ee1211ece3a02004,b533f7e2003418c63fd71471264c559a,sneakers,mobile,2019-04-24 09:19:02,banner_show,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   Unnamed: 0    1000000 non-null  int64 
 1   order_id      1000000 non-null  object
 2   user_id       1000000 non-null  object
 3   page_id       1000000 non-null  object
 4   product       1000000 non-null  object
 5   site_version  1000000 non-null  object
 6   time          1000000 non-null  object
 7   title         1000000 non-null  object
 8   target        1000000 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 68.7+ MB


In [10]:
df.describe(include = ['object'])

,order_id,user_id,page_id,product,site_version,time,title
count,1000000,1000000,1000000,1000000,1000000,1000000,1000000
unique,1000000,882589,959023,5,2,952092,3
top,707c234e742d9bac2ac48112c3b9f1bd,2a98f631f947487fc710b89d13d5a51a,a3d2de7675556553a5f08e4c88d2c228,clothes,mobile,2019-01-14 18:47:47,banner_show
freq,1,8,29395,210996,718521,5,872275


## Some Exploration to Better Understand our Data

Lets's look at the different banner types:

In [11]:
df['product'].value_counts()

clothes             210996
company             203020
sneakers            201298
sports_nutrition    193200
accessories         191486
Name: product, dtype: int64

In [17]:
df.groupby(by = 'product')['target'].value_counts()

product           target
accessories       0         186127
                  1           5359
clothes           0         197720
                  1          13276
company           0         203020
sneakers          0         193375
                  1           7923
sports_nutrition  0         190363
                  1           2837
Name: target, dtype: int64

In [18]:
df.groupby(['product', 'target']).count()

Unnamed: 0  order_id  user_id  page_id  site_version  \
product          target                                                         
accessories      0           186127    186127   186127   186127        186127   
                 1             5359      5359     5359     5359          5359   
clothes          0           197720    197720   197720   197720        197720   
                 1            13276     13276    13276    13276         13276   
company          0           203020    203020   203020   203020        203020   
sneakers         0           193375    193375   193375   193375        193375   
                 1             7923      7923     7923     7923          7923   
sports_nutrition 0           190363    190363   190363   190363        190363   
                 1             2837      2837     2837     2837          2837   

                           time   title  
product          target                  
accessories      0       186127  186127  
                 1         5359    5359  
clothes          0       197720  197720  
                 1        13276   13276  
company          0       203020  203020  
sneakers         0       193375  193375  
                 1         7923    7923  
sports_nutrition 0       190363  190363  
                 1         2837    2837

In [19]:
df.groupby(['product', 'target']).agg('count')

Unnamed: 0  order_id  user_id  page_id  site_version  \
product          target                                                         
accessories      0           186127    186127   186127   186127        186127   
                 1             5359      5359     5359     5359          5359   
clothes          0           197720    197720   197720   197720        197720   
                 1            13276     13276    13276    13276         13276   
company          0           203020    203020   203020   203020        203020   
sneakers         0           193375    193375   193375   193375        193375   
                 1             7923      7923     7923     7923          7923   
sports_nutrition 0           190363    190363   190363   190363        190363   
                 1             2837      2837     2837     2837          2837   

                           time   title  
product          target                  
accessories      0       186127  186127  
                 1         5359    5359  
clothes          0       197720  197720  
                 1        13276   13276  
company          0       203020  203020  
sneakers         0       193375  193375  
                 1         7923    7923  
sports_nutrition 0       190363  190363  
                 1         2837    2837

Let's look at the range of time-stamps on these data:

In [21]:
df['time'].min()

'2019-01-01 00:00:25'

In [22]:
df['time'].max()

'2019-05-31 23:59:21'

Let's check the counts of the different site_version values:

In [23]:
df['site_version'].value_counts()

mobile     718521
desktop    281479
Name: site_version, dtype: int64

And now check titles

In [24]:
df['title'].value_counts()

banner_show     872275
banner_click     98330
order            29395
Name: title, dtype: int64

In [25]:
len(df.loc[df['title']=='order'])

29395

In [26]:
sum(df['target'])

29395

In [27]:
df.groupby('title').agg({'target':'mean'})

,target
title,
banner_click,0
banner_show,0
order,1


## Experimental Setup

We need to filter by site_version, time, and product:

In [34]:
df_AB = df.loc[(df['site_version']=='desktop') & (df['time'] >= '2019-05-01')
              & ((df['product']=='sneakers') | (df['product']=='accessories'))]

In [35]:
df_AB.tail()

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,title,target
999904,5738290,ecfe9da2dbea255f05b2f526a4c663a7,cd7598ba5798ad6bd3f5a667ae4c217a,644ad50dd900d6735d8ceb15b71d50f8,accessories,desktop,2019-05-01 23:01:05,banner_show,0
999908,7885931,8c28b45ff7fbb5d5df0b7670183689db,2f28106a4db2babcfde1c4095c805632,a3d2de7675556553a5f08e4c88d2c228,sneakers,desktop,2019-05-26 11:19:09,order,1
999915,8099155,c613db2f19c7285f101196062c178490,97473368941b89c4716c4f389e9b8bd7,a3d2de7675556553a5f08e4c88d2c228,sneakers,desktop,2019-05-11 09:04:46,order,1
999930,6690689,6f7f32ecf6790e4eb06384b479f43162,73f403b4843c70c41c0db294ee479582,13f395d7c86ef276f7ce52a557fd3491,accessories,desktop,2019-05-09 09:17:24,banner_show,0
999999,7650805,af3a19fcceb477d2992f1771af3a9b34,515cb969971fb5e08ae0316f58e3f434,ef38d906f15db1efde867c68f1336946,sneakers,desktop,2019-05-20 17:58:16,banner_show,0


In [36]:
df_AB['product'].value_counts()

sneakers       13269
accessories    12703
Name: product, dtype: int64

In [37]:
df_AB['site_version'].value_counts()

desktop    25972
Name: site_version, dtype: int64

### What Test Would Make Sense?

Since we're comparing the frequency of conversions of customers who saw the "sneakers" banner against those who saw the "accessories" banner, we can use a $\chi^2$ test.

Note there are other hypothesis tests we can use but this should be fine since it should fit our criteria.

### The Hypotheses

$H_0$: Customers who saw the sneakers banner were no more or less likely to buy than customers who saw the accessories banner.

$H_1$: Customers who saw the sneakers banner were more or less likely to buy than customers who saw the accessories banner.

### Setting a Threshold

We'll set a false-positive rate of $\alpha = 0.05$.

## $\chi^2$ Test

### Setup the Data

We need our contingency table: the numbers of people who did or did not submit orders, both for the accessories banner and the sneakers banner. 

In [44]:
df_AB.groupby(['product', 'target']).count()['title']

product      target
accessories  0         12207
             1           496
sneakers     0         12470
             1           799
Name: title, dtype: int64

In [47]:
counts = df_AB.groupby(['product', 'target']).count()['title']
counts

product      target
accessories  0         12207
             1           496
sneakers     0         12470
             1           799
Name: title, dtype: int64

In [45]:
df_A = df_AB.loc[df_AB['product'] == 'accessories']
df_B = df_AB.loc[df_AB['product'] == 'sneakers']

In [46]:
accessories_orders = sum(df_A['target'])
sneakers_orders = sum(df_B['target'])

accessories_orders, sneakers_orders

(496, 799)

In [49]:
accessories_orders = counts[1]
accessories_orders

496

To get the numbers of people who didn't submit orders, we get the total number of people who were shown banners and then subtract the numbers of people who did make orders.

In [53]:
accessories_total = len(df_A)
sneakers_total = len(df_B)

accessories_no_orders = accessories_total - accessories_orders
sneakers_no_orders = sneakers_total - sneakers_orders

accessories_no_orders, sneakers_no_orders

(12207, 12470)

In [54]:
contingency_table = np.array([[accessories_orders, accessories_no_orders],
                             [sneakers_orders, sneakers_no_orders]])

contingency_table

array([[  496, 12207],
       [  799, 12470]])

In [56]:
contin_list = list([[accessories_orders, accessories_no_orders],
                             [sneakers_orders, sneakers_no_orders]])
contin_list

[[496, 12207], [799, 12470]]

In [57]:
pd.DataFrame([[accessories_orders, accessories_no_orders],
                             [sneakers_orders, sneakers_no_orders]])

,0,1
0,496,12207
1,799,12470


In [58]:
np.array([[45, 45, 45], [30, 36, 45], [60, 70, 80]])

array([[45, 45, 45],
       [30, 36, 45],
       [60, 70, 80]])

### Calculation


In [59]:
stats.chi2_contingency(contingency_table)

(60.94640624666488,
 5.86498985882812e-15,
 1,
 array([[  633.38922686, 12069.61077314],
        [  661.61077314, 12607.38922686]]))

In [60]:
stats.chi2_contingency(pd.DataFrame([[accessories_orders, accessories_no_orders],
                             [sneakers_orders, sneakers_no_orders]]))

(60.94640624666488,
 5.86498985882812e-15,
 1,
 array([[  633.38922686, 12069.61077314],
        [  661.61077314, 12607.38922686]]))

This extremely low $p$-value suggests that these two groups are genuinely performing differently. In particular, the desktop customers who saw the sneakers banner in May 2019 bought at a higher rate than the desktop customers who saw the accessories banner in May 2019.

In [62]:
(sneakers_orders / sneakers_total) * 100

6.021553998040545

In [64]:
(accessories_orders / accessories_total) * 100

3.9045894670550267

## Interpretation

In [65]:
contingency_table

array([[  496, 12207],
       [  799, 12470]])

In [66]:
contingency_table[:, 0] / contingency_table.sum(axis=1)

array([0.03904589, 0.06021554])

In [67]:
contingency_table.sum(axis=1)

array([12703, 13269])

In [68]:
# Find the difference in conversion rate
accessory_CR, sneaker_CR = contingency_table[:, 0] / contingency_table.sum(axis=1)

In [69]:
print(f'Conversion Rate for accessory banner:\n\t{100*accessory_CR:.3f}%')
print(f'Conversion Rate for sneaker banner:\n\t{100*sneaker_CR:.3f}%')
print('')
print(f'Absolute difference of CR: {100*(sneaker_CR-accessory_CR):.3f}%')

Conversion Rate for accessory banner:
	3.905%
Conversion Rate for sneaker banner:
	6.022%

Absolute difference of CR: 2.117%


In [70]:
accessories_total, sneakers_total

(12703, 13269)

In [71]:
accessories_total - sneakers_total

-566

In [72]:
496 / accessories_total * 100

3.9045894670550267

In [73]:
799 / sneakers_total * 100

6.021553998040545

So we can say:
- There was a statistically significant difference at the $\alpha$-level (confidence level)
- The difference was about $2\%$ in favor of the sneaker banner!

# Exercise

> The company is impressed with what you found and is now wondering if there is a difference in their other banner ads!

With your group, look at the same month (May 2019) but compare different platforms ('mobile' vs 'desktop') and or different banner types ('accessories', 'sneakers', 'clothes', 'sports_nutrition'). Just don't repeat the same test we did above 😉

Make sure you record what considerations you have for the experiment, what hypothesis test you performed ($H_0$ and $H_1$ too), and your overall conclusion/interpretation for the _business stakeholders_. Is there a follow-up you'd suggest? 

H0: There is no difference btwn desktop and mobile conversion rates

HA: There is a difference btwn desktop and mobile conversion rates

In [89]:
df.groupby('site_version').count()['target']

site_version
desktop    281479
mobile     718521
Name: target, dtype: int64

In [79]:
df['title'].value_counts()

banner_show     872275
banner_click     98330
order            29395
Name: title, dtype: int64

In [81]:
#create subsets for each category
df_A = df.loc[df['site_version'] == 'desktop']
df_B = df.loc[df['site_version'] == 'mobile']

In [84]:
#get number that did submit orders
desktop_orders = sum(df_A['target'])
mobile_orders = sum(df_B['target'])

desktop_orders, mobile_orders

(15719, 13676)

In [85]:
#get number that didn't submit orders
desktop_total = len(df_A)
mobile_total = len(df_B)

desktop_no_orders = desktop_total - desktop_orders
mobile_no_orders = mobile_total - mobile_orders

desktop_no_orders, mobile_no_orders

(265760, 704845)

In [87]:
#create contingency table
contingency_table = np.array([[desktop_orders, desktop_no_orders],
                            [mobile_orders, mobile_no_orders]])
contingency_table

array([[ 15719, 265760],
       [ 13676, 704845]])

In [88]:
#Perform chi2_contingency test
stats.chi2_contingency(contingency_table)

(9604.192143184011,
 0.0,
 1,
 array([[  8274.075205, 273204.924795],
        [ 21120.924795, 697400.075205]]))

In [92]:
# Find the difference in conversion rate
desktop_CR, mobile_CR = contingency_table[:, 0] / contingency_table.sum(axis=1)

print(f'Conversion Rate for desktop:\n\t{100*desktop_CR:.3f}%')
print(f'Conversion Rate for mobile:\n\t{100*mobile_CR:.3f}%')
print('')
print(f'Absolute difference of CR: {100*(desktop_CR-mobile_CR):.3f}%')

Conversion Rate for desktop:
	5.584%
Conversion Rate for mobile:
	1.903%

Absolute difference of CR: 3.681%


We are rejecting H0 b/c our P Value is below alpha. This means that there is a difference between Desktop and Mobile orders. Specifically, Desktop receives more orders than Mobile.